## Assignment 10 - Assembly of the System Vector and Boundary Conditions
In this assignment we want to perform the first step in assembling the linear system arising from the Finite Element discretization of a partial differential equation. For this task we want to use the work we have done implementing <i>Grid, Basis</i> and <i>Quadrature</i>.<br>
Consider the Convection-Diffusion-Problem 

$$ -\nabla\cdot(a \nabla u)  + \mathbf{b}\nabla u + cu = f  \text{ in }\Omega$$

with Dirichlet boundary conditions

$$ u \equiv g \text{ on }\delta\Omega $$


where $a,c,f\colon\Omega\longrightarrow\mathbb{R}$ and $\mathbf{b}\colon\Omega\longrightarrow\mathbb{R}^2$

For a given grid $\tau_h$ with $N_{DOF}$ points and global basis functions $\lbrace \varphi_i \rbrace_{i=1}^{N_{DOF}}$ we have seen that the discrete weak form of the problem is given by <br> <br>
<center>Find $U_j\in \mathbb{R}$ for $j=1,\dots,N_{DOF}$ such that $U_j=g(x_j)$ on the boundary and for inner DOFs
$$ \sum\limits_{j=1}^{N_{DOF}} U_j \underbrace{\left( a\int_\Omega \nabla \varphi_j \cdot \nabla \varphi_i
+b\int_\Omega \mathbf{v}\cdot\nabla\varphi_j\varphi_i 
+c\int_\Omega \varphi_j\varphi_i \right) }_{=A_{ij}}
= \underbrace{\int_\Omega f \varphi_i}_{l_i}$$
holds for all $i=1,\dots,N_{innerDOF}$.</center> <br>
This problem is a linear system that can be solved for the coefficients $U_j$ representing the solution values at all degrees of freedom - a DOF vector. In this assignment we want to focus on the system vector $l$, i.e. the right-hand side of the linear system. We use the knowledge from previous assignments and recall the following notation:<ul> 
<li> $\tau_h$ - the grid, i.e. the set of all cells</li>
<li> $\hat{T}$ - the reference cell </li>
<li> $\varphi_i$ - global basis function with index $i$ </li>
<li> $\phi_j^T,\hat{\phi}_j$ - local basis function with index $j$ on $T$ or $\hat{T}$ resp. </li>
<li> $\mathcal{D}_T$ - DOF map of cell $T$ (assignment 9)</li>
<li> $F_T,DF_T$ - the reference map of cell $T$ and its Jacobian(assignment 4)</li>
<li> $\lbrace(\hat{w}_k,\hat{x}_k)\rbrace_{k=1}^{N_Q}$ - a quadrature rule on $\hat{T}$</li>
<li> $supp(\varphi_i)$ - the support, i.e. all cells where $\varphi_i$ is not the constant $0$</li> 
</ul>
Then we remember the following derivation on how we can practically compute $l_i$:
$$ l_i = \int_\Omega f(x) \varphi_i(x) \mathrm{d}x$$ 
Integrate and summing over the triangles in the support the global basis function $\varphi_i$:
$$ = \sum\limits_{T \in supp(\varphi_i)} \int_T f(x) \varphi_i(x) \mathrm{d}x$$
Use the correspinding local basis function:
$$= \sum\limits_{T \in supp(\varphi_i)} \int_T f(x) \phi_{\mathcal{D}_T(i)}^T(x) \mathrm{d}x$$
Transform the integral from the local triangle to reference element:
$$=\sum\limits_{T \in supp(\varphi_i)} |\det D F_T| \int_{\hat{T}} f (F_T(\hat{x})) \hat{\phi}_{\mathcal{D}_T(i)}(\hat{x}) \mathrm{d} \hat{x}$$
Apply a quadrature rule to discretise the integral:
$$\approx\sum\limits_{T \in supp(\phi_i)}|\det D F_T| \sum\limits_{k=1}^{N_Q} \hat{w}_k f (F_T(\hat{x}_k)) \hat{\phi}_{\mathcal{D}_T(i)}(\hat{x}_k)$$
For each global index $i$ this is a sum over a few grid cells - usually the ones surrounding point $i$ - and a small number of quadrature points. Values of the right-hand side source $f$ at transformed quadrature points are multiplied with evaluations of the reference basis function $\hat{\phi}_{\mathcal{D}_T(i)}$ and scaled with both the quadrature weight $\hat{w}_k$ and the area scaling factor $|\det D F_T|$.

### 10.1 Preparation

Download the python file <i>classes.py</i> and <b>place them in the same folder where this notebook is being executed</b>. Make yourself familiar with the classes and their methods. You know all of them from the previous assignments, but new functionality was added. The <b>TriGrid</b>, <b>Basis</b> and <b>Quadrature</b> are imported from the python file. Prepare the assembling by:
<ul> 
    <li> discretise the domain $[0,1]^2$ by a triangular grid called <b>grid</b> with $N_x,N_y = 4$.</li>
    <li> generate a linear basis object called <b>basis</b> </li>
    <li> generate a quadrature object of exactness $2$ called <b>quadrature</b> </li>
    <li> define a right side source $f(x,y) = x + y$ by a lambda function </li>
    <li> define boundary values $g(x,y) = x$ by a lambda function </li>    
</ul>

In [37]:
import numpy as np
from classes import TriGrid,Basis,Quadrature

### BEGIN SOLUTION
grid = TriGrid(0,1,0,1,4,4)
quadrature = Quadrature(2)
basis = Basis()
f = lambda x,y: x+y
g = lambda x,y: np.zeros_like(x)
### END SOLUTION

In [38]:
### BEGIN HIDDEN TESTS
assert np.all(grid.points == TriGrid(0,1,0,1,4,4).points)
assert np.all(grid.cells == TriGrid(0,1,0,1,4,4).cells)
### END HIDDEN TESTS

In [39]:
### BEGIN HIDDEN TESTS
assert np.all(abs(quadrature.points - Quadrature(2).points)<1e-5)
assert np.all(abs(quadrature.weights - Quadrature(2).weights)<1e-5)
### END HIDDEN TESTS

In [40]:
### BEGIN HIDDEN TESTS
grid = TriGrid(0,1,0,1,4,4)
assert np.all(abs(f(grid.points[:,0],grid.points[:,1]) - (grid.points[:,0]+grid.points[:,1]))<1e-5) 
assert np.all(abs(g(grid.points[:,0],grid.points[:,1]))<1e-5 )
### END HIDDEN TESTS

### 10.2 StationaryProblem Class
The StationaryProblem object represents a Convetion-Diffusion Problem as given above and it will handles the discretization by finite elements. In this taks we will assemble the system vector and boundary conditions. 

<b>10.2.1 The Constructor</b><br>
In the code block below the class constructor is given. Understand the code. 

<b>10.2.2 The Boundary Conditions</b><br>
Write the function <b>assembleBoundaryCondtions</b> with the following input arguments:

<b>INPUT:</b> $dirichletValues$ - lambda function 

For the boundary DOFs this function should fill the Dirichlet Values in the system vector and it should fill the coresponding elements in the diagonal of the system matrix with ones. 

<b>10.2.3 The system vector</b><br>
Write a function <b>addSource</b> which adds the values of the right hand site function $f$ to the system vector. 

<b>INPUT:</b> $f$ - lambda function

<b>Outlook</b><br> 
In the next assignemnt we will assmble the full system matrix and solve the linear system. 

In [41]:
class StationaryProblem():
  def __init__(self, grid, basis, quadrature, dirichletLocations = (lambda x,y: True), dirichletValues = (lambda x,y: 0.0)):
    self.grid = grid
    self.basis = basis
    self.quadrature = quadrature

    # Initialize 
    self.systemMatrix = np.zeros((np.shape(self.grid.points)[0],np.shape(self.grid.points)[0]))
    self.systemVector = np.zeros(np.shape(self.grid.points)[0])
    self.solution = None

    # Boundary Conditions
    self.dirichletDOFs = self.grid.getBoundaryIndices(dirichletLocations)
    self.allDOFs = np.arange(np.shape(self.grid.points)[0])
    self.freeDOFs = np.setdiff1d(np.arange(np.shape(self.grid.points)[0]),self.dirichletDOFs)    
    self.assembleBoundaryConditions(dirichletValues)

    # Precompute data that is needed all the time.
    # x_k and w_k
    self.xkHat,self.wkHat = self.quadrature.getPointsAndWeights()
    # F_T(x_k)
    self.xkTrafo = self.grid.evalReferenceMap(self.xkHat)
    # DF_T -> det and inverse
    self.dets = np.abs(self.grid.getDeterminants())
    self.invJac = self.grid.getInverseJacobians()
    # phi(x_k) and grad_phi(x_k)  
    self.phi = self.basis.evalPhi(self.xkHat)
    self.gradPhi = self.basis.evalGradPhi(self.xkHat)
        
  ### BEGIN SOLUTION
  def assembleBoundaryConditions(self, dirichletValues):
    self.systemMatrix[self.dirichletDOFs,self.dirichletDOFs] = 1.0
    self.systemVector[self.dirichletDOFs] = dirichletValues(self.grid.points[self.dirichletDOFs,0],self.grid.points[self.dirichletDOFs,1])
        
  def addSource(self, f):
    for i in self.freeDOFs:  
      supp,localInd = self.grid.evalDOFMap(i)
      for T,loc_i in zip(supp,localInd):
        self.systemVector[i] += self.dets[T] * np.sum(self.phi[:,loc_i] * self.wkHat * f(self.xkTrafo[T,:,0],self.xkTrafo[T,:,1]))
                                             
  ### END SOLUTION

In [42]:
pde = StationaryProblem(grid,basis,quadrature,lambda x,y:True,g)
pde.addSource(f)
print(pde.systemVector)

[0.         0.         0.         0.         0.         0.07407407
 0.11111111 0.         0.         0.11111111 0.14814815 0.
 0.         0.         0.         0.        ]


In [43]:
### BEGIN HIDDEN TESTS
g = TriGrid(0,1,0,1,4,4)
q = Quadrature(2)
b = Basis()
f = lambda x,y: x+y
g = lambda x,y: x
pde = StationaryProblem(grid,basis,quadrature,lambda x,y:True,g)
assert np.max(abs(pde.systemVector[pde.dirichletDOFs] - grid.points[pde.dirichletDOFs,0]))<1e-5
### END HIDDEN TESTS

In [44]:
### BEGIN HIDDEN TESTS
g = TriGrid(0,1,0,1,4,4)
q = Quadrature(2)
b = Basis()
f = lambda x,y: x+y
g = lambda x,y: x
pde = StationaryProblem(grid,basis,quadrature,lambda x,y:True,g)
pde.addSource(f)
assert np.max(abs(pde.systemVector[pde.freeDOFs] - np.array([0.07407407 ,  0.11111111 , 0.11111111, 0.14814815  ]))) < 1e-5
### END HIDDEN TESTS

In [45]:
### BEGIN HIDDEN TESTS
g = TriGrid(0,1,0,1,4,4)
q = Quadrature(2)
b = Basis()
f = lambda x,y: x+y
g = lambda x,y: x
pde = StationaryProblem(grid,basis,quadrature,lambda x,y:True,g)
pde.addSource(f)
assert abs(np.sum(pde.systemMatrix) - 12) < 1e-5
assert np.max(abs(pde.systemMatrix[pde.dirichletDOFs,pde.dirichletDOFs] - 1 )) < 1e-5
### END HIDDEN TESTS